In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import xgboost as xgb
from xgboost import XGBRegressor
import joblib
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score

In [2]:
data = pd.read_csv('datos.csv')
data

/var/folders/d_/k0sktyy51_v9gfr8ch8hd9bh0000gn/T/ipykernel_14103/4029292871.py:1: DtypeWarning: Columns (13,19,21,22,23,27,33) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('datos.csv')


,@timestamp,external_id,post_date,make,model,trimlevel,car_year,fueltype,transmission,km,...,region,car_name,url,seller_gallery_url,seller_phone_number,damages,color,misc,folder_index,location
0,2023-03-11T06:51:38,1084339020,2023-03-10,Fiat,Egea,1.4 Fire Street,2019,Benzin,Manuel,110000.0,...,Gaziemir,RCN MOTORS'DAN FIAT EGE HB 1.4FIRE STREET 95HP,https://www.sahibinden.com/ilan/vasita-otomobi...,https://rcnmotors.sahibinden.com,0 (538) 325 47 58,NaN,Beyaz,NaN,20230311_d247f56e-9016-49de-bc84-b5f675befb2e,NaN
1,2023-03-11T06:51:45,1084296114,2023-03-10,Fiat,Egea,1.4 Fire Street,2022,Benzin,Manuel,14200.0,...,Sancaktepe,iCAR'DAN %18 FATURALI 1.4 FIRE STREET EGEA HAT...,https://www.sahibinden.com/ilan/vasita-otomobi...,https://icar.sahibinden.com,0 (533) 025 35 36,NaN,Beyaz,NaN,20230311_d247f56e-9016-49de-bc84-b5f675befb2e,NaN
2,2023-03-11T06:51:50,1083218091,2023-03-05,Nissan,Qashqai,1.5 dCi Sky Pack,2015,Dizel,Manuel,103000.0,...,Alasehir,SG MOTORS 2015 QASHQAI 1.5 SKYPACK CAM TAVANLI...,https://www.sahibinden.com/ilan/vasita-arazi-s...,https://sgmotorss.sahibinden.com,0 (236) 654 44 42,NaN,Beyaz,NaN,20230311_d247f56e-9016-49de-bc84-b5f675befb2e,NaN
3,2023-03-11T06:51:52,1082848718,2023-03-04,Renault,Symbol,1.5 DCI Joy,2014,Dizel,Manuel,180000.0,...,Cekmekoy,temiz sembol,https://www.sahibinden.com/ilan/vasita-otomobi...,/ilan-sahibine-soru-sor/1082848718,0 (545) 510 53 61,NaN,Beyaz,NaN,20230311_d247f56e-9016-49de-bc84-b5f675befb2e,NaN
4,2023-03-11T06:51:55,1084320432,2023-03-10,Fiat,Egea,1.3 Multijet Easy,2020,Dizel,Manuel,74461.0,...,Zeytinburnu,HEDEF RENAULT PLAZA'DAN %18 KDV'LI 2020 FIAT E...,https://www.sahibinden.com/ilan/vasita-otomobi...,https://hedefotomotivrenault.sahibinden.com,0 (212) 546 94 00,NaN,Beyaz,NaN,20230311_d247f56e-9016-49de-bc84-b5f675befb2e,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509995,2022-03-16T12:50:41,1008804311,2022-03-15T00:00:00,Mercedes - Benz,C Serisi,C 180 AMG 9G-Tronic,2017,Benzin,Otomatik,95500.0,...,Beylikduzu,sahibinde acill hatasiz,https://www.sahibinden.com/ilan/vasita-otomobi...,NaN,0 (532) 601 51 16,NaN,Siyah,NaN,20220316_71492cf3-1744-4df7-9d51-07decb553012,Istanbul
1509996,2022-03-16T12:50:44,1008695096,2022-03-15T00:00:00,Volkswagen,Jetta,1.2 TSI BlueMotion Comfortline,2016,Benzin,Manuel,123000.0,...,Karsiyaka,"Temiz, hasar kayitsiz 2016 Jetta 1.2 BMT Comfo...",https://www.sahibinden.com/ilan/vasita-otomobi...,NaN,0 (532) 377 13 66,NaN,Beyaz,NaN,20220316_71492cf3-1744-4df7-9d51-07decb553012,Izmir
1509997,2022-03-16T12:50:47,1008986891,2022-03-16T00:00:00,Audi,A6,A6 Sedan 2.0 TDI Quattro,2017,Dizel,Yarı Otomatik,107000.0,...,Silivri,SAHIBINDEN HATASIZ BOYASIZ TRAMERSIZ AUDI A6 S...,https://www.sahibinden.com/ilan/vasita-otomobi...,NaN,0 (534) 067 70 79,NaN,Beyaz,NaN,20220316_71492cf3-1744-4df7-9d51-07decb553012,Istanbul
1509998,2022-03-16T12:50:48,1007835552,2022-03-16T00:00:00,Peugeot,301,1.6 HDi Active,2017,Dizel,Manuel,107500.0,...,Zara,SAHIBINDEN 2017 92HP ACTIVE 107.XXXKM COK TEMIZ,https://www.sahibinden.com/ilan/vasita-otomobi...,NaN,0 (553) 841 08 58,NaN,Beyaz,NaN,20220316_71492cf3-1744-4df7-9d51-07decb553012,Sivas


In [3]:
data['car_age'] = pd.to_datetime('today').year - data['car_year']
data.drop(['car_year'], axis=1, inplace=True)
data['horse_power'] = data['horse_power'].str.extract('(\d+)').astype(float)

In [4]:
columns_of_interest = ['make', 'model', 'km', 'fueltype', 'transmission', 'horse_power', 'car_age', 'price']
data = data[columns_of_interest]

In [5]:
def remove_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
    return data

data = remove_outliers(data, 'price')
data = remove_outliers(data, 'km')
data = remove_outliers(data, 'car_age')

In [6]:
data.dropna(inplace=True)

data.shape

data.to_csv('../webpage/datos_procesados.csv', index=False)

In [18]:
X = data.drop('price', axis=1)
y = data['price']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [20]:
categorical_features = ['make', 'model', 'fueltype', 'transmission']
numeric_features = ['km', 'horse_power', 'car_age']

In [21]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [22]:
modelos = [
    ('XGB Regressor', XGBRegressor(objective='reg:squarederror')),
    ('Gradient Boosting Regressor', GradientBoostingRegressor()),
    ('K-nearest Neighbors Regressor', KNeighborsRegressor()),
    #('Random Forest Regressor', RandomForestRegressor()),
]

In [25]:
parametros = {
    'K-Nearest Neighbors': {'regressor__n_neighbors': [5, 10, 15, 20]},
}

In [26]:
for name, params in parametros.items():
    for n_neighbors in params['regressor__n_neighbors']:
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('regressor', KNeighborsRegressor(n_neighbors=n_neighbors))])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        print(f"Modelo: {name}, Vecinos: {n_neighbors}")
        print('Coeficiente de determinación (R^2):', r2)
        print('Error absoluto medio (MAE):', mae)
        print('\n')

Modelo: K-Nearest Neighbors, Vecinos: 5
Coeficiente de determinación (R^2): 0.8613889905016296
Error absoluto medio (MAE): 58004.632573787596


Modelo: K-Nearest Neighbors, Vecinos: 10
Coeficiente de determinación (R^2): 0.8699293596860193
Error absoluto medio (MAE): 56240.195044588436




KeyboardInterrupt: 

In [36]:
resultados = []
for nombre, modelo in modelos:
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', modelo)])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    resultados.append((nombre, mae, r2))
    print(f'{nombre} - Error Absoluto Medio: {mae}, R2 Score: {r2}')

XGB Regressor - Error Absoluto Medio: 60058.59873405982, R2 Score: 0.8586882224023054
Gradient Boosting Regressor - Error Absoluto Medio: 73527.72965957753, R2 Score: 0.7937418432289671
K-nearest Neighbors Regressor - Error Absoluto Medio: 58004.632573787596, R2 Score: 0.8613889905016296


In [26]:
# guardar xgbregressor
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', XGBRegressor())])

pipeline.fit(X, y)

y_pred = pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print(f'Error Absoluto Medio: {mae}, R2 Score: {r2}')

Error Absoluto Medio: 59853.62993355811, R2 Score: 0.8599547922819029


In [28]:
import pickle

with open('../webpage/modelo.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [29]:
import pickle

with open('../webpage/modelo.pkl', 'rb') as f:
    model = pickle.load(f)

print(type(model))

<class 'sklearn.pipeline.Pipeline'>
